In [7]:
import os
import re
import cv2
import numpy as np
from ultralytics import YOLO
from skimage import exposure
from typing import Tuple
import logging

logging.getLogger("ultralytics").setLevel(logging.ERROR)

# ========= VARIÁVEIS DE CONFIGURAÇÃO =========
INPUT_FOLDER = "/media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms/article/analisar"  # ✅ Altere aqui
MODEL_PATH = "/media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms/yolo-train/eggs-scanner-image.v2i.yolov11/runs/detect/train2/weights/best-train2.onnx"  # ✅ Altere aqui
WINDOW_SIZE = 254
CONFIDENCE_THRESHOLD = 0.5
IOU_THRESHOLD = 0.8
# ============================================

PATTERN = re.compile(r"(.*)\.(png|jpg|jpeg)$", re.IGNORECASE)

def iou(boxA, boxB):
    xA = max(boxA[0], boxB[0])
    yA = max(boxA[1], boxB[1])
    xB = min(boxA[2], boxB[2])
    yB = min(boxA[3], boxB[3])
    interArea = max(0, xB - xA) * max(0, yB - yA)
    boxAArea = (boxA[2] - boxA[0]) * (boxA[3] - boxA[1])
    boxBArea = (boxB[2] - boxB[0]) * (boxB[3] - boxB[1])
    unionArea = boxAArea + boxBArea - interArea
    return interArea / unionArea if unionArea > 0 else 0

def filter_overlapping_boxes(boxes, threshold=IOU_THRESHOLD):
    filtered = []
    for box in boxes:
        if all(iou(box, kept) < threshold for kept in filtered):
            filtered.append(box)
    return filtered

class ImageProcessor:
    def __init__(self, model: YOLO, window_size: int = WINDOW_SIZE, confidence_threshold: float = CONFIDENCE_THRESHOLD) -> None:
        self.window_size = window_size
        self.model = model
        self.confidence_threshold = confidence_threshold

    def load_image(self, image_path: str) -> np.ndarray:
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Não foi possível carregar a imagem: {image_path}")
        return image

    def normalize_square(self, square: np.ndarray) -> np.ndarray:
        return exposure.adjust_gamma(square, gamma=1.5)

    def process_image(self, image_path: str, output_folder: str) -> None:
        image = self.load_image(image_path)
        img_height, img_width, channels = image.shape
        padded_height = ((img_height + self.window_size - 1) // self.window_size) * self.window_size
        padded_width = ((img_width + self.window_size - 1) // self.window_size) * self.window_size
        padded_image = np.zeros((padded_height, padded_width, channels), dtype=image.dtype)
        padded_image[:img_height, :img_width, :] = image
        annotated_image = padded_image.copy()

        all_boxes = []

        for y in range(0, padded_height, self.window_size):
            for x in range(0, padded_width, self.window_size):
                window = padded_image[y:y + self.window_size, x:x + self.window_size]
                processed_window = self.normalize_square(window)
                results = self.model(processed_window, verbose=False)
                boxes = results[0].boxes

                if boxes and boxes.xyxy is not None:
                    for box in boxes:
                        score = float(box.conf[0]) if box.conf is not None else 0.0
                        if score < self.confidence_threshold:
                            continue
                        x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
                        global_box = [x + x1, y + y1, x + x2, y + y2]
                        all_boxes.append(global_box)

        final_boxes = filter_overlapping_boxes(all_boxes)

        for box in final_boxes:
            x1, y1, x2, y2 = box
            cv2.rectangle(annotated_image, (x1, y1), (x2, y2), (0, 255, 0), 2)
            label = "egg"
            cv2.putText(annotated_image, label, (x1, max(y1 - 10, 10)),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        final_image = annotated_image[:img_height, :img_width]

        base_name, ext = os.path.splitext(os.path.basename(image_path))
        output_name = f"{base_name} - IA{ext}"
        output_path = os.path.join(output_folder, output_name)
        cv2.imwrite(output_path, final_image)
        print(f"✅ Processada: {output_name} — Objetos finais: {len(final_boxes)}")


def load_model_safe(model_path: str) -> YOLO:
    try:
        model = YOLO(model_path)
        dummy = np.zeros((256, 256, 3), dtype=np.uint8)
        model(dummy)
        return model
    except Exception as e:
        if model_path.endswith(".onnx"):
            print(f"⚠️ Erro ao carregar modelo ONNX: {e}")
            fallback_path = model_path.replace(".onnx", ".pt")
            print(f"🔁 Tentando carregar fallback: {fallback_path}")
            return YOLO(fallback_path)
        else:
            raise e


def main():
    print(f"📂 Processando imagens em: {INPUT_FOLDER}")
    os.makedirs(INPUT_FOLDER, exist_ok=True)

    model = load_model_safe(MODEL_PATH)
    processor = ImageProcessor(model)

    image_files = [f for f in os.listdir(INPUT_FOLDER) if PATTERN.match(f)]

    for filename in sorted(image_files):
        full_path = os.path.join(INPUT_FOLDER, filename)
        try:
            processor.process_image(full_path, INPUT_FOLDER)
        except Exception as e:
            print(f"❌ Erro ao processar {filename}: {e}")


if __name__ == "__main__":
    main()


📂 Processando imagens em: /media/williancaddd/CODES/WORKSPACE-FIOTEC/eggs-count-algorithms/article/analisar


2025-05-13 15:16:33.381675640 [W:onnxruntime:, transformer_memcpy.cc:83 ApplyImpl] 4 Memcpy nodes are added to the graph main_graph for CUDAExecutionProvider. It might have negative impact on performance (including unable to run CUDA graph). Set session_options.log_severity_level=1 to see the detail logs before this message.


✅ Processada: paleta-1-25 - IA.jpg — Objetos finais: 16
✅ Processada: paleta-10-4 - IA.jpg — Objetos finais: 4
✅ Processada: paleta-140-5 - IA.jpg — Objetos finais: 6
✅ Processada: paleta-146-4 - IA.jpg — Objetos finais: 5
✅ Processada: paleta-16-4 - IA.jpg — Objetos finais: 3
✅ Processada: paleta-17-8 - IA.jpg — Objetos finais: 15
✅ Processada: paleta-2-2 - IA.jpg — Objetos finais: 2
✅ Processada: paleta-2-4 - IA.jpg — Objetos finais: 7
✅ Processada: paleta-203-1 - IA.jpg — Objetos finais: 1
✅ Processada: paleta-268-3 - IA.jpg — Objetos finais: 4
✅ Processada: paleta-285-1 - IA.jpg — Objetos finais: 3
✅ Processada: paleta-33-12 - IA.jpg — Objetos finais: 10
✅ Processada: paleta-340-9 - IA.jpg — Objetos finais: 19
✅ Processada: paleta-4-11 - IA.jpg — Objetos finais: 19
✅ Processada: paleta-4-5 - IA.jpg — Objetos finais: 7
✅ Processada: paleta-5-20 - IA.jpg — Objetos finais: 26
✅ Processada: paleta-50-13 - IA.jpg — Objetos finais: 22
✅ Processada: paleta-9-137 - IA.jpg — Objetos finais: